In [ ]:
import os 
import time
import uuid 
import cv2
import random

In [ ]:
uuid.uuid1()

In [ ]:
IMAGES_PATH = os.path.join('D:\\data', 'images')  # final path: D:\images\images
number_images = 60
if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH)

# Annotate Images with LabelMe

# Review Dataset and Build Image Loading

In [ ]:
cap = cv2.VideoCapture(0)

# Check if camera opened
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

for imgnum in range(number_images):
    print('Collecting image{}'.format(imgnum))
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame!")
        continue  # Skip this iteration

    imgname = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:

!labelme


### Partition augmentaiton data

In [ ]:
import tensorflow as tf
import numpy as np
import json 
import matplotlib.pyplot as plt

Limit Gpu Memory Growth

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu , True)

In [ ]:
tf.config.list_physical_devices()

Load Image into Tf data Pipeline

In [ ]:
images = tf.data.Dataset.list_files('D:/data/images/*.jpg', shuffle=True)


In [ ]:
images.as_numpy_iterator().next()

view Raw images with matplotlib



In [ ]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img, channels=3)
    return img

In [ ]:
images = images.map(load_image)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
image_generator = images.batch(4).as_numpy_iterator()

In [ ]:

plot_images = image_generator.next()

Partition Augemntaed data


In [ ]:
len(images)

Manuallu split data into train test and val

Make the directory

In [ ]:

base_dir = 'D:/data'

In [ ]:
import os
import shutil



# Create necessary folders if they don't exist
for folder in ['train', 'test', 'val']:
    image_folder = os.path.join(base_dir, folder, 'images')
    label_folder = os.path.join(base_dir, folder, 'labels')
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(label_folder, exist_ok=True)


Shuffle the images 

In [ ]:
all_images = os.listdir(os.path.join(base_dir, 'images'))
random.shuffle(all_images)  # Shuffle for randomness

Split the data and define data size

In [ ]:
# Split the data
# Calculate split sizes
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

total_images = len(all_images)
train_count = int(total_images * train_ratio)
val_count = int(total_images * val_ratio)

In [ ]:
train_files = all_images[:train_count]
val_files = all_images[train_count:train_count + val_count]
test_files = all_images[train_count + val_count:]

Function to move file in train test and val data

In [ ]:
import os
import shutil

def copy_files(file_list, split_name):
    for file in file_list:
        filename_wo_ext = os.path.splitext(file)[0] + '.json'
        
        # Paths for image
        src_image = os.path.join(base_dir, 'images', file)
        dst_image = os.path.join(base_dir, split_name, 'images', file)
        
        # Copy image (always)
        shutil.copy2(src_image, dst_image)
        
        # Paths for label
        src_label = os.path.join(base_dir, 'labels', filename_wo_ext)
        dst_label = os.path.join(base_dir, split_name, 'labels', filename_wo_ext)
        
        # Copy label only if exists
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)
        else:
            print(f"Label missing for {file}, only image copied.")

# Copy files to respective folders
copy_files(train_files, 'train')
copy_files(val_files, 'val')
copy_files(test_files, 'test')

# Apply Image Augmentaion

In [ ]:

import albumentations as alb

In [ ]:

augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

In [ ]:

img = cv2.imread(os.path.join(base_dir,'train', 'images','3f096c72-4f84-11f0-9d23-f43bd8788907.jpg'))

In [ ]:

with open(os.path.join(base_dir, 'train', 'labels', '3f096c72-4f84-11f0-9d23-f43bd8788907.json'), 'r') as f:
    label = json.load(f)

In [ ]:
img.shape

In [ ]:
label

In [ ]:
type(label)

In [ ]:
label['shapes'][0]['points']

# Extract Coordinates and Resclae to Match Image Resolution

In [ ]:
#As Batch Size = 4 so need corrdinates of 4 images
coords= [0,0,0,0]
coords[0] = label['shapes'][0]['points'][0][0]
coords[1] = label['shapes'][0]['points'][0][1]
coords[2] = label['shapes'][0]['points'][1][0]
coords[3] = label['shapes'][0]['points'][1][1]

In [ ]:
coords

In [ ]:
#Scalind
coords = list(np.divide(coords, [640,480,640,480]))

In [ ]:
coords

# Apply Augmentaion and View Results

In [ ]:
augmented = augmentor(image = img, bboxes=[coords], class_labels = ['face'])

In [ ]:
augmented.keys()


In [ ]:
augmented['bboxes']

Build and run augmentaion pipeline

In [ ]:
cv2.rectangle(augmented['image'],
             tuple(np.multiply(augmented['bboxes'][0][:2], [450, 450]).astype(int)),
             tuple(np.multiply(augmented['bboxes'][0][2:], [450, 450]).astype(int)),
              (250,250,0),2)
plt.imshow(augmented['image'])

# Step 5 Build and Run Augmentaion Pipeline


5.1 Run augmentaion Pipeline

In [ ]:
# Create necessary folders if they don't exist
for folder in ['train', 'test', 'val']:
    image_folder = os.path.join('D:/aug_data', folder, 'images')
    label_folder = os.path.join('D:/aug_data', folder, 'labels')
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(label_folder, exist_ok=True)

In [ ]:
for partition in ['train', 'test', 'val']:
    for image in os.listdir(os.path.join(base_dir, partition, 'images')):
        img = cv2.imread(os.path.join(base_dir, partition,'images', image))
       
        coords = [0,0,0.00001, 0.00001] #for if label doesn't exist
        label_path = os.path.join(base_dir, partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path,'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, (640,480,640,480)))

        try:
            for x in range(60): #creating 60 augmented image for single image
                augmented =augmentor(image = img ,bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('D:/aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if(len(augmented['bboxes'])  == 0):
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0
                    else:
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1

                else :
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0
                with open(os.path.join('D:/aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)
        except Exception as e:
            print(e)

5.2 Load Augmented images to tensorflow dataset

In [ ]:
train_images = tf.data.Dataset.list_files("D:/aug_data/train/images/*.jpg" , shuffle = False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files("D:/aug_data/test/images/*.jpg" , shuffle = False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files("D:/aug_data/val/images/*.jpg" , shuffle = False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

# 6 Prepare Labels

6.1 Build Label Loading Function

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = 'utf-8') as f:
        label = json.load(f)
    return [label['class']], label['bbox']

6.2 Load Label nto tensorflow dataset

In [ ]:
train_labels = tf.data.Dataset.list_files("D:/aug_data/train/labels/*.json",shuffle = False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x],[tf.uint8, tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files("D:/aug_data/test/labels/*.json",shuffle = False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x],[tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files("D:/aug_data/val/labels/*.json",shuffle = False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x],[tf.uint8, tf.float16]))

In [ ]:
train_labels.as_numpy_iterator().next()

# 7 Combined Load and Image sample

7.1 Check Partition Lengths

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

7.2 Create final Datasets (images/labelx)

In [ ]:

train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(5000)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(5000)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:

train.as_numpy_iterator().next()[1]

7.3 View Images and Annotations

In [ ]:

data_samples = train.as_numpy_iterator()


In [ ]:
res =data_samples.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx].copy()
    sample_coords = res[1][1][idx]
    
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                        (255,0,0), 2)

    ax[idx].imshow(sample_image)

# Build Deep Learning using the Fuctional Api

8.1 Import Layers and Base Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

### Download VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
vgg.summary()

### 8.3 Build instance of network

In [ ]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  for detect object is presnet or not
    f1 = GlobalMaxPooling2D()(vgg)  #going from all layers of vgg
    class1 = Dense(2048, activation='relu')(f1) 
    class2 = Dense(1, activation='sigmoid')(class1)
    
    # Bounding box model for predicting the rectangel box coordinates
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

In [ ]:

facetracker = build_model()

In [ ]:

facetracker.summary()

In [ ]:
train.as_numpy_iterator().next()[1] #As two ouptut classes and Coordinates

In [ ]:

X, y = train.as_numpy_iterator().next()

In [ ]:

X.shape 

In [ ]:
y[0].shape, y[1].shape

In [ ]:

classes, coords = facetracker.predict(X)

In [ ]:

classes, coords

# 9 Define Losses and Optimizers

### 9.1 Define Optimizer and Lr

In [ ]:

batches_per_epoch = len(train)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [ ]:
lr_decay #learning reate

In [ ]:

opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)


### 9.2 Create Localization Loss and Classificatio Loss

In [ ]:
def localization_loss(y_true, yhat):
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))

    h_true = y_true[:,3]-y_true[:,1]
    w_true = y_true[:,2]-y_true[:,0]

    h_pred = yhat[:,3]- yhat[:,1]
    w_pred = yhat[:,2]- yhat[:,0]

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred)+ tf.square(h_true - h_pred))

    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss

### 9.3 Test Our Loss Metrics

In [ ]:
localization_loss(y[1], coords).numpy()

In [ ]:
classloss(y[0], classes)

In [ ]:

regressloss(y[1], coords)

# Train Neural Network

In [ ]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss

            #doing gradients and back proopogation
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)


In [ ]:

model = FaceTracker(facetracker)

In [ ]:

model.compile(opt, classloss, regressloss)

### 10.2 Train

In [ ]:
tf.config.list_physical_devices()

In [ ]:

logdir='logs'

In [ ]:

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
tensorboard_callback

In [ ]:
hist = model.fit(train, epochs=40, validation_data=val, callbacks=[tensorboard_callback])

### 10.3 Plot performance

In [ ]:

hist.history

In [ ]:

fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

# Make Predictions

### 11.1  Make Predictions on Test Set

In [ ]:

test_data = test.as_numpy_iterator()

In [ ]:

test_sample = test_data.next()

In [ ]:

yhat = facetracker.predict(test_sample[0])

In [ ]:

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx].copy()
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx] > 0.5:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)

# 11.2 Save the model

In [ ]:

from tensorflow.keras.models import load_model

In [ ]:
model_version = 3

In [ ]:

facetracker.save(fr'D:/models/faceTracker/{model_version}')

In [ ]:
facetracker = load_model(fr'D:/models/faceTracker/{model_version}')

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()